In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
os.getcwd()

'/app'

In [4]:
os.listdir()

['.ipynb_checkpoints', 'graphs', 'main.ipynb', 'requirements.txt']

In [5]:
from graphs.testing import test

In [8]:
test()

Hello World
